In [3]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
from collections import deque
from sklearn.preprocessing import StandardScaler
import time

In [4]:
with open('trainset.pkl', 'rb') as f:
    X_train = pickle.load(f)

# Create and fit the StandardScaler using the training features.
scaler = StandardScaler()
scaler.fit(X_train)

# Load the trained model.
with open('model_red_rf.pkl', 'rb') as f:
    model = pickle.load(f)

# Label mapping (ensure this matches your model's encoding)
label_map = {0: "Push Ups", 1: "Pull Ups", 2: "Squats", 3: "Jumping Jacks", 4: "Planks"}

In [5]:
# Define the angle calculation function.
def calculate_angle(a, b, c):
    """Calculate the angle (in degrees) at point b given three points."""
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle


In [6]:
def get_point(landmark, w, h):
    """Convert normalized landmark to pixel coordinates."""
    return [int(landmark.x * w), int(landmark.y * h)]

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# ---------------------
# Timing Parameters:
# measurement_duration: time window for angle data collection (20 sec).
# display_duration: time to display the prediction result after measurement (7 sec).
measurement_duration = 20  # seconds
display_duration = 7       # seconds

phase = "measurement"   # current phase: either "measurement" or "display"
phase_start_time = time.time()  # starting time for current phase

# Storage for accumulated angle data.
angle_data = {
    'shoulder': [],
    'elbow': [],
    'hip': [],
    'knee': [],
    'ankle': []
}
predicted_label = None  # variable to store prediction from measurement phase

# ---------------------
# Open the webcam and set up a resizable window.
# ---------------------
cap = cv2.VideoCapture(0)
cv2.namedWindow('MediaPipe Pose', cv2.WINDOW_NORMAL)
cv2.resizeWindow('MediaPipe Pose', 1280, 720)

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Unable to read from webcam. Exiting...")
            break

        h, w, _ = frame.shape
        # Convert the frame from BGR to RGB.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image.
        results = pose.process(image)

        # Convert back to BGR.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw pose landmarks.
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            landmarks = results.pose_landmarks.landmark
            
            # Extract left-side landmark points.
            left_shoulder = get_point(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], w, h)
            left_elbow = get_point(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value], w, h)
            left_wrist = get_point(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value], w, h)
            left_hip = get_point(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value], w, h)
            left_knee = get_point(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value], w, h)
            left_ankle = get_point(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value], w, h)
            left_foot = get_point(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value], w, h)
            
            # Calculate the five joint angles.
            shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            ankle_angle = calculate_angle(left_knee, left_ankle, left_foot)
            
            # Draw instantaneous angle text (for debugging, optional)
            inst_angle_text = (f"S:{int(shoulder_angle)} E:{int(elbow_angle)} "
                               f"H:{int(hip_angle)} K:{int(knee_angle)} A:{int(ankle_angle)}")
            cv2.putText(image, inst_angle_text, (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2, cv2.LINE_AA)
            
            # Phase: Measurement
            if phase == "measurement":
                # Accumulate angles.
                angle_data['shoulder'].append(shoulder_angle)
                angle_data['elbow'].append(elbow_angle)
                angle_data['hip'].append(hip_angle)
                angle_data['knee'].append(knee_angle)
                angle_data['ankle'].append(ankle_angle)
            
            # Otherwise, if phase == "display", nothing new is accumulated.
        
        # Check phase timing.
        elapsed = time.time() - phase_start_time
        
        if phase == "measurement" and elapsed >= measurement_duration:
            # Time's up for measurement phase.
            avg_shoulder = np.mean(angle_data['shoulder']) if angle_data['shoulder'] else 0
            avg_elbow = np.mean(angle_data['elbow']) if angle_data['elbow'] else 0
            avg_hip = np.mean(angle_data['hip']) if angle_data['hip'] else 0
            avg_knee = np.mean(angle_data['knee']) if angle_data['knee'] else 0
            avg_ankle = np.mean(angle_data['ankle']) if angle_data['ankle'] else 0

            feature_vector = np.array([avg_shoulder, avg_elbow, avg_hip, avg_knee, avg_ankle]).reshape(1, -1)
            feature_vector_scaled = scaler.transform(feature_vector)
            # Predict using the model.
            pred = model.predict(feature_vector_scaled)[0]
            predicted_label = label_map.get(pred, "Unknown")
            
            # Display average angles (for debugging) and prediction on the image.
            avg_text = (f"Avg Angles (20s): S:{int(avg_shoulder)} E:{int(avg_elbow)} "
                        f"H:{int(avg_hip)} K:{int(avg_knee)} A:{int(avg_ankle)}")
            cv2.putText(image, avg_text, (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
            
            # Switch to display phase.
            phase = "display"
            phase_start_time = time.time()  # Reset timer for display phase.
        
        elif phase == "display" and elapsed >= display_duration:
            # Once display phase is over, reset for next measurement window.
            phase = "measurement"
            phase_start_time = time.time()
            angle_data = {key: [] for key in angle_data}  # clear accumulator

        # If in display phase, overlay the predicted label.
        if phase == "display" and predicted_label is not None:
            pred_text = f"Prediction: {predicted_label}"
            cv2.putText(image, pred_text, (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Show the resized output image.
        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\Tzur\miniconda3\envs\pythonProject\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Tzur\miniconda3\envs\pythonProject\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Tzur\miniconda3\envs\pythonProject\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Tzur\miniconda3\envs\pythonProject\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


KeyboardInterrupt: 

In [35]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define the mapping (if needed) for numeric labels to string labels
label_map = {0: "Push Ups", 1: "Pull Ups", 2: "Squats", 3: "Jumping Jacks", 4: "Planks"}

# -------------------------------
# 1. Load the trained model.
# -------------------------------
with open('model_red_rf.pkl', 'rb') as f:
    model = pickle.load(f)

# -------------------------------
# 2. Load the training set to fit the scaler.
#    The pickle file is assumed to contain a tuple: (X_train, y_train)
# -------------------------------
with open('trainset.pkl', 'rb') as f:
    X_train= pickle.load(f)

# -------------------------------
# 3. Load the test set.
#    The pickle file is assumed to contain a tuple: (X_test, y_test)
# -------------------------------
with open('y_testset.pkl', 'rb') as f:
    y_test = pickle.load(f)
    
with open('testset.pkl', 'rb') as f:
    X_test = pickle.load(f)


# -------------------------------
# 4. Fit the StandardScaler on the training data and transform the test set.
# -------------------------------
scaler = StandardScaler()
scaler.fit(X_train)
X_test_scaled = scaler.transform(X_test)

# -------------------------------
# 5. Get predictions and prediction probabilities.
# -------------------------------
pred = model.predict(X_test_scaled)
pred_prob = model.predict_proba(X_test_scaled)

# Optionally, map numeric predictions to string labels.
predicted_labels = [label_map.get(p, "Unknown") for p in pred]
true_labels = [label_map.get(y, "Unknown") for y in y_test]

# -------------------------------
# 6. Compare predictions to true labels.
# -------------------------------
accuracy = accuracy_score(y_test, pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("\nClassification Report:")
print(classification_report(y_test, pred, target_names=list(label_map.values())))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, pred))

# Optionally, print a few sample predictions vs. true labels.
print("\nSample Predictions:")
for i in range(min(10, len(y_test))):
    print(f"Sample {i+1}: True Label: {true_labels[i]}, Predicted Label: {predicted_labels[i]}, Probabilities: {pred_prob[i]}")

Accuracy: 94.97%

Classification Report:
               precision    recall  f1-score   support

     Push Ups       0.96      0.95      0.95       782
     Pull Ups       0.93      0.94      0.93       999
       Squats       0.97      0.99      0.98      1464
Jumping Jacks       0.97      0.92      0.95       661
       Planks       0.92      0.90      0.91       749

     accuracy                           0.95      4655
    macro avg       0.95      0.94      0.95      4655
 weighted avg       0.95      0.95      0.95      4655


Confusion Matrix:
[[ 739   30    3    3    7]
 [  13  942   17    5   22]
 [   2    2 1452    2    6]
 [   3   11   13  611   23]
 [  13   33   19    7  677]]

Sample Predictions:
Sample 1: True Label: Squats, Predicted Label: Squats, Probabilities: [0.   0.08 0.77 0.02 0.13]
Sample 2: True Label: Squats, Predicted Label: Squats, Probabilities: [0. 0. 1. 0. 0.]
Sample 3: True Label: Squats, Predicted Label: Squats, Probabilities: [0.   0.01 0.88 0.   0.11]